In [153]:
#!pip install veragridengine
#!pip3 install torch torchvision
#!pip install pandapower
#!pip install torch_geometric
#!pip install networkx
#!pip install seaborn
#Use a new virtual environment
# the venv requires pytorch

In [154]:
# Import necessary libraries for data manipulation, plotting, and network analysis
import networkx as nx  # For handling graph data structures
import numpy as np  # For numerical operations
import pandas as pd  # For data manipulation using DataFrames
import logging  # For logging messages
import random  # For generating random numbers
import warnings
import os
import sys
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import json

#import GridCalEngine.api as gce  # For interfacing with the GridCal API
#import GridCalEngine.Devices as dev
#import GridCalEngine.Simulations as sim
#from GridCalEngine.Compilers.circuit_to_newton_pa import translate_newton_pa_pf_results, newton_pa_pf
#from GridCalEngine.IO.file_handler import FileOpen
#import GridCalEngine.enumerations as en

import VeraGridEngine.api as gce  # For interfacing with the GridCal API
import VeraGridEngine.Devices as dev
import VeraGridEngine.Simulations as sim
from VeraGridEngine.Compilers.circuit_to_newton_pa import translate_newton_pa_pf_results, newton_pa_pf
from VeraGridEngine.IO.file_handler import FileOpen
import VeraGridEngine.enumerations as en

import pandapower as pp
import simbench as sb
import pandapower.topology as top  # For topology analysis in Pandapower
import pandapower.plotting as plot  # For plotting in Pandapower
import pandapower.networks as nw

import src.GC_PandaPowerImporter as GC_PandaPowerImporter

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
warnings.filterwarnings("ignore")

## Basics

In [155]:
#open a matpower/matlab file 

#VeraGrid formats : .gridcal, .xls (special), .m (matpower/matlab)  : cases from matpower in github

grid = FileOpen(".\\networks\\case118.m").open()
grid.name = "case118"

VeraGrid networks are data structures which model the network and are composed by different elements :
- buses
- lines
- loads
- generators
- transformers
- shunts
- others

In [156]:
#list main grid features
print("grid :")
print("name:",grid.name)
print("Sbase:",grid.Sbase)
print("number of buses:",len(grid.buses))
print("number of lines:",len(grid.lines))
print("number of loads:",len(grid.loads))
print("number of generators:",len(grid.generators))
print("number of transformers:",len(grid.transformers2w))
print("number of shunts:",len(grid.shunts))

grid :
name: case118
Sbase: 100.0
number of buses: 118
number of lines: 186
number of loads: 118
number of generators: 54
number of transformers: 0
number of shunts: 2


The coder needs to know the attributes on each data type (consulting VeraGrid code)

all GridCal elements are identified by their idtag, which is a string, i.e 99314938a5604c2aaa04f87815618341

the name of the element is only for human readability, and it is not unique.

the idtag is a hash of the name, and it is unique for each element in the grid.

you need the idtag to make relations between elements (e.g. line to bus, transformer to bus, etc.), we will see an example later in this notebook

In [157]:
##List the first 5 lines and some of their parameters
for line in grid.lines[:5]:
    print(f"LINE id:{line.idtag} name:{line.name}, length:{line.length}, buses:{line.bus_from}-{line.bus_to}, impendance:{line.R:.3f}+j{line.X:.3f}, active:{line.active} ")

LINE id:a2d31a59271f41f1b8ebbd2d065bf99d name:1_2_1, length:1.0, buses:1-2, impendance:0.030+j0.100, active:True 
LINE id:fd6b2c3c92704c5e9dcfc795079ba3bf name:1_3_1, length:1.0, buses:1-3, impendance:0.013+j0.042, active:True 
LINE id:cfb7c3e599fc4c2db646ffe1d554326c name:4_5_1, length:1.0, buses:4-5, impendance:0.002+j0.008, active:True 
LINE id:7404572245044fde9cc60a19791f8294 name:3_5_1, length:1.0, buses:3-5, impendance:0.024+j0.108, active:True 
LINE id:fc9a3d9ee7ac4c06881f522215f95f16 name:5_6_1, length:1.0, buses:5-6, impendance:0.012+j0.054, active:True 


In [158]:
line_name = [line.name for line in grid.lines]
print(line_name)

['1_2_1', '1_3_1', '4_5_1', '3_5_1', '5_6_1', '6_7_1', '8_9_1', '8_5_1', '9_10_1', '4_11_1', '5_11_1', '11_12_1', '2_12_1', '3_12_1', '7_12_1', '11_13_1', '12_14_1', '13_15_1', '14_15_1', '12_16_1', '15_17_1', '16_17_1', '17_18_1', '18_19_1', '19_20_1', '15_19_1', '20_21_1', '21_22_1', '22_23_1', '23_24_1', '23_25_1', '26_25_1', '25_27_1', '27_28_1', '28_29_1', '30_17_1', '8_30_1', '26_30_1', '17_31_1', '29_31_1', '23_32_1', '31_32_1', '27_32_1', '15_33_1', '19_34_1', '35_36_1', '35_37_1', '33_37_1', '34_36_1', '34_37_1', '38_37_1', '37_39_1', '37_40_1', '30_38_1', '39_40_1', '40_41_1', '40_42_1', '41_42_1', '43_44_1', '34_43_1', '44_45_1', '45_46_1', '46_47_1', '46_48_1', '47_49_1', '42_49_1', '42_49_1', '45_49_1', '48_49_1', '49_50_1', '49_51_1', '51_52_1', '52_53_1', '53_54_1', '49_54_1', '49_54_1', '54_55_1', '54_56_1', '55_56_1', '56_57_1', '50_57_1', '56_58_1', '51_58_1', '54_59_1', '56_59_1', '56_59_1', '55_59_1', '59_60_1', '59_61_1', '60_61_1', '60_62_1', '61_62_1', '63_59_1',

In [159]:
for line in grid.lines[:5]:
    print(line.name, line.bus_from, line.bus_to, line.R, line.X, line.B, line.rate, line.active, line.length)


1_2_1 1 2 0.0303 0.0999 0.0254 10000.0 True 1.0
1_3_1 1 3 0.0129 0.0424 0.01082 10000.0 True 1.0
4_5_1 4 5 0.00176 0.00798 0.0021 10000.0 True 1.0
3_5_1 3 5 0.0241 0.108 0.0284 10000.0 True 1.0
5_6_1 5 6 0.0119 0.054 0.01426 10000.0 True 1.0


In [160]:
#Create an array of the tuples (R,X) of the lines
impendances = [(line.R,line.X) for line in grid.lines]
#and display the first 5
impendances[:5]

[(0.0303, 0.0999),
 (0.0129, 0.0424),
 (0.00176, 0.00798),
 (0.0241, 0.108),
 (0.0119, 0.054)]

In [161]:
##List the first 5 buses and some of their parameters
for bus in grid.buses[:5]:
    print(f"BUS id:{bus.idtag} name:{bus.name}, vnom={bus.Vnom}, is slack bus:{bus.is_slack}")

BUS id:7af3124fe2b547679246fdfa9948d9df name:1, vnom=138.0, is slack bus:True
BUS id:ad1b4ef8fb6e48c191035a246ab37159 name:2, vnom=138.0, is slack bus:False
BUS id:013d46fe26f44df8818cd8c608423256 name:3, vnom=138.0, is slack bus:False
BUS id:012925126b4b48caa1d6e6e26db89818 name:4, vnom=138.0, is slack bus:False
BUS id:e2f2861f8f744f7eba2448e4d92e0982 name:5, vnom=138.0, is slack bus:False


In [162]:
##List the first 5 loads and some of their parameters
for load in grid.loads[:5]:
    print(f"LOAD: id:{load.idtag} name:{load.name}, bus={load.bus.name}:{load.bus.idtag}, P={load.P}, Q={load.Q}")

LOAD: id:e39a6635d3e34a17b2884dd2dcb85b85 name:Load@1, bus=1:7af3124fe2b547679246fdfa9948d9df, P=51.0, Q=27.0
LOAD: id:d1a0cd42e31f496dab69cbc1962f63da name:Load@2, bus=2:ad1b4ef8fb6e48c191035a246ab37159, P=20.0, Q=9.0
LOAD: id:873563619213491fa1c5bc055b919811 name:Load@3, bus=3:013d46fe26f44df8818cd8c608423256, P=39.0, Q=10.0
LOAD: id:04a3cb6593ed471da4627fe1abf4249f name:Load@4, bus=4:012925126b4b48caa1d6e6e26db89818, P=39.0, Q=12.0
LOAD: id:68e499110e434f7186ba03d20b7d9e1a name:Load@5, bus=5:e2f2861f8f744f7eba2448e4d92e0982, P=1e-06, Q=0.0


In [163]:
#array with all the active powers
allP = [load.P for load in grid.loads]
print(allP)

[51.0, 20.0, 39.0, 39.0, 1e-06, 52.0, 19.0, 28.0, 1e-06, 1e-06, 70.0, 47.0, 34.0, 14.0, 90.0, 25.0, 11.0, 60.0, 45.0, 18.0, 14.0, 10.0, 7.0, 13.0, 1e-06, 1e-06, 71.0, 17.0, 24.0, 1e-06, 43.0, 59.0, 23.0, 59.0, 33.0, 31.0, 1e-06, 1e-06, 27.0, 66.0, 37.0, 96.0, 18.0, 16.0, 53.0, 28.0, 34.0, 20.0, 87.0, 17.0, 17.0, 18.0, 23.0, 113.0, 63.0, 84.0, 12.0, 12.0, 277.0, 78.0, 1e-06, 77.0, 1e-06, 1e-06, 1e-06, 39.0, 28.0, 1e-06, 1e-06, 66.0, 1e-06, 12.0, 6.0, 68.0, 47.0, 68.0, 61.0, 71.0, 39.0, 130.0, 1e-06, 54.0, 20.0, 11.0, 24.0, 21.0, 1e-06, 48.0, 1e-06, 163.0, 10.0, 65.0, 12.0, 30.0, 42.0, 38.0, 15.0, 34.0, 42.0, 37.0, 22.0, 5.0, 23.0, 38.0, 31.0, 43.0, 50.0, 2.0, 8.0, 39.0, 1e-06, 68.0, 6.0, 8.0, 22.0, 184.0, 20.0, 33.0]


In [164]:
##List the generators and some of their parameters
for generator in grid.generators[:5]:
    print(f"Generator: id:{generator.idtag} name:{generator.name}, bus={generator.bus.name}:{generator.bus.idtag}, P={generator.P}, Pmax={generator.Pmax}, Pmin={generator.Pmin}")

Generator: id:14a8b027aac4475eaeee8860a053cfc9 name:, bus=1:7af3124fe2b547679246fdfa9948d9df, P=0.0, Pmax=100.0, Pmin=0.0
Generator: id:4a7ef3d3136644b497ad8d4f0d11b4a9 name:, bus=4:012925126b4b48caa1d6e6e26db89818, P=0.0, Pmax=100.0, Pmin=0.0
Generator: id:3d781792e6174353bf022822573c6d0e name:, bus=6:7479b42f62fd4bd5afc0b3beecee303a, P=0.0, Pmax=100.0, Pmin=0.0
Generator: id:993fce9873214eac886fb4ea41c8fad1 name:, bus=8:4563562d3e344d44ba5979c11343d18d, P=0.0, Pmax=100.0, Pmin=0.0
Generator: id:cdda5aa642c44b918adfc7758f422a6a name:, bus=10:19a469f3a8814ba38a7cf1d6dd323110, P=450.0, Pmax=550.0, Pmin=0.0


In [165]:
## Another way to do read the data, in this case the branches (=lines and transformers)
print(grid.get_branches()[0].bus_to)
print(grid.get_branches()[0].bus_from)
print(grid.get_branches()[0].active)
print(grid.get_branches()[0].R)
print(grid.lines[0].R)
print(grid.get_branches()[0].X)

2
1
True
0.0303
0.0303
0.0999


Per unit: Vbase, Sbase & Zbase

pu = Engineering units / base

In [166]:
grid.Sbase

100.0

In [167]:
Vbase = [bus.Vnom for bus in grid.buses]
Vbase[:5]

[138.0, 138.0, 138.0, 138.0, 138.0]

In [168]:
#search for lines which are not active
[line.name for line in grid.lines if not line.active]

[]

In [169]:
list_buses = [bus.name for bus in grid.buses]
print(list_buses)

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118']


In [170]:
[line.name for line in grid.lines if (line.bus_from.name=='2') or (line.bus_to.name=='2') ]

['1_2_1', '2_12_1']

## Network Creation and Modification

Function which creates a network and returns it

In [171]:
## Function which creates a network and returns it
def CreateGrid(Vbase, Sbase):
    m_circuit = gce.MultiCircuit()
    m_circuit.Sbase = Sbase
    m_circuit.name = 'Test Circuit'
    b1 = gce.Bus('b1', Vnom=Vbase, is_slack=True)
    b2 = gce.Bus('b2', Vnom=Vbase)
    b3 = gce.Bus('b3', Vnom=Vbase)
    m_circuit.add_bus(b1)
    m_circuit.add_bus(b2)
    m_circuit.add_bus(b3)
    
    br0 = gce.Line(b1, b2, name='l1')
    br0.fill_design_properties(r_ohm=0.01, x_ohm=0.03, c_nf=0.0,
                               length=1.0, Imax=1.0, freq=m_circuit.fBase, Sbase=m_circuit.Sbase)
    br1 = gce.Line(b1, b3, name='l2')
    br1.fill_design_properties(r_ohm=0.02, x_ohm=0.05, c_nf=0.0,
                               length=1.0, Imax=1.0, freq=m_circuit.fBase, Sbase=m_circuit.Sbase)
    br2 = gce.Line(b2, b3, name='l3')
    br2.fill_design_properties(r_ohm=0.03, x_ohm=0.08, c_nf=0.0,
                               length=1.0, Imax=1.0, freq=m_circuit.fBase, Sbase=m_circuit.Sbase)
    #add one line with ohms, I need to check how to do it with p.u. Example without using fill_design_properties
    #br2 = gce.Line(b2,b3, name='l3', r=0.03, x=0.8, b=0, length=1.0)

    m_circuit.add_line(br0)
    m_circuit.add_line(br1)
    m_circuit.add_line(br2)

    load1 = gce.Load('load 1'	, P=50, Q=30	)
    load2 = gce.Load('load 2'	, P=150, Q=80	)
    m_circuit.add_load(b2 ,load1)
    m_circuit.add_load(b3 ,load2)


    return m_circuit

In [172]:
#Grid creation
Vbase = 10 #kV
Sbase = 100
grid2 = CreateGrid(Vbase = 10, Sbase = 100)
print("name:",grid2.name)
print("Sbase:",grid2.Sbase)
print("number of buses:",len(grid2.buses))
print("number of lines:",len(grid2.lines))
print("number of loads:",len(grid2.loads))
print("number of generators:",len(grid2.generators))
print("number of transformers:",len(grid2.transformers2w))

name: Test Circuit
Sbase: 100
number of buses: 3
number of lines: 3
number of loads: 2
number of generators: 0
number of transformers: 0


In [173]:
## search the idtag of the bus b2
for bus in grid2.buses:
    if bus.name == 'b2':
        B2 = bus
        break

print(B2.name,B2.idtag)

b2 2cf0e5de0f304277994b6157a300b646


In [174]:
## search for the lines which are connected to b2
for line in grid2.lines:
    if line.bus_from.name == 'b2' or line.bus_to.name == 'b2':
        print(line.name,line.idtag)

l1 587a294f9ae84e8dbf0b1fd4f09556b6
l3 1f3502a544624abda667b6fbc8e5e23e


In [175]:
[line.idtag for line in grid2.lines if (line.bus_from.name=='b2') or (line.bus_to.name=='b2') ]

['587a294f9ae84e8dbf0b1fd4f09556b6', '1f3502a544624abda667b6fbc8e5e23e']

In [176]:
#create a list with the lines which are connected to b2, then the list elements can be used 
list_b2 = [line for line in grid2.lines if line.bus_from.name == 'b2' or line.bus_to.name == 'b2']
print(list_b2)
print(list_b2[0].R)

[.::587a294f9ae84e8dbf0b1fd4f09556b6::l1, .::1f3502a544624abda667b6fbc8e5e23e::l3]
0.01


In [177]:
## search for the lines which are connected between b2 and b3
for line in grid2.lines:
    if (line.bus_from.name == 'b2' and line.bus_to.name == 'b3') or (line.bus_from.name == 'b3' and line.bus_to.name == 'b2') :
        print(line.name,line.idtag)

l3 1f3502a544624abda667b6fbc8e5e23e


In [178]:
## modify the active power for the load connected to b2
print([load.P for load in grid2.loads])

for load in grid2.loads:
    if load.bus.name == 'b2':
        load.P = 100.0
        break

print([load.P for load in grid2.loads])

[50.0, 150.0]
[100.0, 150.0]


In [179]:
#creates a new bus B4 and a new line between B2 (found before, as we do not have its reference) and B4 (which reference is obtained when creating it)
b4 = gce.Bus('b4', Vnom=Vbase)
grid2.add_bus(b4)

newline = gce.Line(b4, B2, name='l4')
newline.fill_design_properties(r_ohm=0.01, x_ohm=0.03, c_nf=0.0, length=1.0, Imax=1.0, freq=grid2.fBase, Sbase=grid2.Sbase)
grid2.add_line(newline)


print("number of buses:",len(grid2.buses))
print("number of lines:",len(grid2.lines))

number of buses: 4
number of lines: 4


In [180]:
##Removes a bus and a line from the grid
grid2.delete_bus(B2)
grid2.delete_line(newline)

print("number of buses:",len(grid2.buses))
print("number of lines:",len(grid2.lines))

number of buses: 3
number of lines: 3


In [181]:
grid.lines[3].active=False
grid.lines[3].active

False

## Power Flow

In [182]:
def runpp(grid): #execute power flow analysis
    options = gce.PowerFlowOptions(gce.SolverType.NR, verbose=False)
    power_flow = gce.PowerFlowDriver(grid, options)
    power_flow.run()
    return power_flow.results

res = runpp(grid)

In [183]:
#converged ?
print(f'Converged: {res.converged.__bool__()}, error: {res.error}')
print(f"Losses with all lines closed are {res.losses.sum().real:.4f} MW, minimal voltage:{res.get_bus_df().Vm.min():.3f}")
print(f"load 10: {grid.loads[10].P}")

Converged: True, error: 1.5652353441142353e-09
Losses with all lines closed are 136.2356 MW, minimal voltage:0.943
load 10: 70.0


In [184]:
grid.loads[10].P *= 2.0
    
options = gce.PowerFlowOptions(gce.SolverType.NR, verbose=False)
power_flow = gce.PowerFlowDriver(grid, options)
power_flow.run()

print('Converged:', power_flow.results.converged, 'error:', power_flow.results.error)
print(f"Losses with all lines closed are {power_flow.results.losses.sum().real:.4f} MW, minimal voltage:{power_flow.results.get_bus_df().Vm.min():.3f}")
print(f"load 10: {grid.loads[10].P}")

Converged: True error: 2.0832476160803637e-09
Losses with all lines closed are 133.9772 MW, minimal voltage:0.943
load 10: 140.0


In [185]:
grid.lines[2].active = False
    
options = gce.PowerFlowOptions(gce.SolverType.NR, verbose=False)
power_flow = gce.PowerFlowDriver(grid, options)
power_flow.run()

print('Converged:', power_flow.results.converged, 'error:', power_flow.results.error)
print(f"Losses with all lines closed are {power_flow.results.losses.sum().real:.4f} MW, minimal voltage:{power_flow.results.get_bus_df().Vm.min():.3f}")
print(f"load 10: {grid.loads[10].P}")

Converged: True error: 2.7527087276268958e-09
Losses with all lines closed are 138.8328 MW, minimal voltage:0.943
load 10: 140.0


In [186]:
##display the node measurements (voltage, angle) in a pandas dataframe
power_flow.results.get_bus_df()

,Vm,Va,P,Q
1,0.955000,0.000000,22.432788,-24.970566
2,0.971471,-0.206934,-20.000000,-9.000000
3,0.955405,-0.696510,-39.000000,-10.000000
4,0.998000,-1.980669,-39.000000,33.680441
5,1.009621,6.980395,-0.000001,0.000000
6,0.990000,2.767297,-52.000000,-16.582749
7,0.989346,1.703438,-19.000000,-2.000000
8,1.015000,11.679653,-28.000000,-75.081453
9,1.042918,18.933758,-0.000001,0.000000
10,1.050000,26.514667,449.999999,-51.042152


In [187]:
#reading the bus voltage/power  results in another way (only the first five lines)

for idx,bus in enumerate(grid.buses[:5]):
    print(bus.name, power_flow.results.voltage[idx], power_flow.results.Sbus[idx])

1 (0.955+0j) (22.43278775752553-24.970565960335275j)
2 (0.9714644572875965-0.003508636881958194j) (-19.999999915227303-8.999999724729127j)
3 (0.9553344486118127-0.011614000512248576j) (-38.99999994596372-9.99999984204458j)
4 (0.99740373974882-0.03449318679200471j) (-38.99999999998555+33.68044111841384j)
5 (1.0021377653301957+0.12269898253000426j) (-9.998489709107994e-07+3.544266824082411e-10j)


In [188]:
## display the line&transformer (both are branches) measurements (current, power) in a pandas dataframe
power_flow.results.get_branch_df()

,Pf,Qf,Pt,Qt,loading,Ploss,Qloss
1_2_1,-1.300080,-16.503249,1.378871,14.406183,-0.013001,0.078791,-2.097066
1_3_1,23.732868,-8.467317,-23.644206,7.771501,0.237329,0.088661,-0.695816
4_5_1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3_5_1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5_6_1,138.447056,10.450761,-136.194793,-1.656007,1.384471,2.252263,8.794754
6_7_1,84.194793,-14.926742,-83.852751,15.938036,0.841948,0.342042,1.011294
8_9_1,-440.635011,-89.733626,445.254648,24.428906,-4.406350,4.619637,-65.304720
8_5_1,314.436491,33.349057,-314.436491,-7.436932,3.144365,0.000000,25.912125
9_10_1,-445.254649,-24.428906,449.999999,-51.042152,-4.452546,4.745350,-75.471058
4_11_1,-39.000000,33.680441,39.569663,-33.520130,-0.390000,0.569663,0.160311


In [189]:
#reading the line current results in another way (only the first five lines)

for idx,line in enumerate(grid.lines[:5]):
    print(line.name, power_flow.results.If[idx], power_flow.results.It[idx], power_flow.results.losses[idx])

1_2_1 (-0.0136134033736659+0.17280889204579175j) (0.013657963062066791-0.1483427934382373j) (0.07879064481239273-2.0970661416659873j)
1_3_1 (0.24851170449958815+0.08866300282682715j) (-0.24844887275681682-0.07832809345983449j) (0.08866134513126411-0.6958156387889165j)
4_5_1 0j 0j 0j
3_5_1 0j 0j 0j
5_6_1 (1.3736926670315412+0.06390646487907503j) (-1.374908302519966-0.04971075406912462j) (2.252262636910416+8.794754206067257j)


## Graphs

In [190]:
#Creates a Networkx graph (or multigraph) from the grid, but contains all edges=lines, even if they are not active
graph = grid.build_graph()

print(len(graph.nodes)) #number of nodes
print(len(graph.edges)) #number of edges

118
186


In [191]:
graph.nodes(data=True)

NodeDataView({0: {}, 1: {}, 2: {}, 3: {}, 4: {}, 5: {}, 6: {}, 7: {}, 8: {}, 9: {}, 10: {}, 11: {}, 12: {}, 13: {}, 14: {}, 15: {}, 16: {}, 17: {}, 18: {}, 19: {}, 20: {}, 21: {}, 22: {}, 23: {}, 24: {}, 25: {}, 26: {}, 27: {}, 28: {}, 29: {}, 30: {}, 31: {}, 32: {}, 33: {}, 34: {}, 35: {}, 36: {}, 37: {}, 38: {}, 39: {}, 40: {}, 41: {}, 42: {}, 43: {}, 44: {}, 45: {}, 46: {}, 47: {}, 48: {}, 49: {}, 50: {}, 51: {}, 52: {}, 53: {}, 54: {}, 55: {}, 56: {}, 57: {}, 58: {}, 59: {}, 60: {}, 61: {}, 62: {}, 63: {}, 64: {}, 65: {}, 66: {}, 67: {}, 68: {}, 69: {}, 70: {}, 71: {}, 72: {}, 73: {}, 74: {}, 75: {}, 76: {}, 77: {}, 78: {}, 79: {}, 80: {}, 81: {}, 82: {}, 83: {}, 84: {}, 85: {}, 86: {}, 87: {}, 88: {}, 89: {}, 90: {}, 91: {}, 92: {}, 93: {}, 94: {}, 95: {}, 96: {}, 97: {}, 98: {}, 99: {}, 100: {}, 101: {}, 102: {}, 103: {}, 104: {}, 105: {}, 106: {}, 107: {}, 108: {}, 109: {}, 110: {}, 111: {}, 112: {}, 113: {}, 114: {}, 115: {}, 116: {}, 117: {}})

In [192]:
#our own function to convert a grid into a graph
def GC2graph(grid):
    G = nx.Graph()

    # Add nodes (buses) to the graph
    for bus in grid.buses:
        G.add_node(bus.idtag, voltage=bus.Vnom)

    # Add edges (lines) to the graph
    for line in grid.lines:
        if line.active:
            G.add_edge(line.bus_from.idtag, line.bus_to.idtag, r=line.R, x=line.X, length=line.length, type="line")
    for trafo in grid.transformers2w:
        if trafo.active:
            G.add_edge(trafo.bus_from.idtag, trafo.bus_to.idtag, r=trafo.R, x=trafo.X, type="trafo")

    return G

In [193]:
graph = GC2graph(grid)
cycles = nx.cycle_basis(graph)

In [194]:
graph.edges(data=True)

EdgeDataView([('7af3124fe2b547679246fdfa9948d9df', 'ad1b4ef8fb6e48c191035a246ab37159', {'r': 0.0303, 'x': 0.0999, 'length': 1.0, 'type': 'line'}), ('7af3124fe2b547679246fdfa9948d9df', '013d46fe26f44df8818cd8c608423256', {'r': 0.0129, 'x': 0.0424, 'length': 1.0, 'type': 'line'}), ('ad1b4ef8fb6e48c191035a246ab37159', '9f76f3b8831b4faabc7e13facfdced71', {'r': 0.0187, 'x': 0.0616, 'length': 1.0, 'type': 'line'}), ('013d46fe26f44df8818cd8c608423256', '9f76f3b8831b4faabc7e13facfdced71', {'r': 0.0484, 'x': 0.16, 'length': 1.0, 'type': 'line'}), ('012925126b4b48caa1d6e6e26db89818', '3ef522546ef445f797d4db84387b3287', {'r': 0.0209, 'x': 0.0688, 'length': 1.0, 'type': 'line'}), ('e2f2861f8f744f7eba2448e4d92e0982', '7479b42f62fd4bd5afc0b3beecee303a', {'r': 0.0119, 'x': 0.054, 'length': 1.0, 'type': 'line'}), ('e2f2861f8f744f7eba2448e4d92e0982', '4563562d3e344d44ba5979c11343d18d', {'r': 0.0, 'x': 0.0267, 'length': 1.0, 'type': 'line'}), ('e2f2861f8f744f7eba2448e4d92e0982', '3ef522546ef445f797d4db8

In [195]:
## using networkx you can analyze the graph
#i.e. is it connected ? or how many loops are there ?
print("connected:",nx.is_connected(graph)) #True or False
print("loops",nx.cycle_basis(graph)) 

connected: True
loops [['3f34668bbe144eedab628584f567c5a0', '0deffad97ca146c4bde9834b401fc0df', 'def84cdd3dcf4472ba0d2b03b378082c', '5f585eb7a3d047b89ab6db3762dfbc43'], ['723f1434a11e4eb0a2f3fffc77a97341', '00cb36559ba443fbab102dba178cf3ea', '5c449b97b92c469d89846fa81d33cf14', '0deffad97ca146c4bde9834b401fc0df'], ['723f1434a11e4eb0a2f3fffc77a97341', 'e9ceca3310bc4f29b67a340674e167ef', '00cb36559ba443fbab102dba178cf3ea'], ['102efc37719a4033a66e6663ecd72944', 'be7f54a896a74efea1cb8f25a7c48dba', '00cb36559ba443fbab102dba178cf3ea'], ['16145c8f6d1347b1867199d7a42e80cb', '38be25f02226417fb5443718c96025e7', '102efc37719a4033a66e6663ecd72944'], ['670e4784d4f940ce9b506a8a2dd3ea19', '2ab097f8c83d41caab4edfd1faf39f7f', 'fac4c0661b184ee6bf7d5d1f80a0a203'], ['a5fdf464e92843d78ddc65d3a5345522', '670e4784d4f940ce9b506a8a2dd3ea19', 'fac4c0661b184ee6bf7d5d1f80a0a203', '38be25f02226417fb5443718c96025e7'], ['5d829d984d61403aaa7565e2905b8493', '670e4784d4f940ce9b506a8a2dd3ea19', 'fac4c0661b184ee6bf7d5d1f8

# PandaPower to VeraGrid/GridCal

In [196]:
gridPP=nw.case33bw()

pp.runpp(gridPP)
print("powerflow of the original pandapower network in pandapower")
print("   bus -5: ", gridPP.res_bus.tail(4))
#print("   line -5: ", gridPP.res_line.tail(1))
print("   power losses:", gridPP.res_line.pl_mw.sum(), gridPP.res_line.ql_mvar.sum())

gridGC = GC_PandaPowerImporter.PP2GC(gridPP)

for line in gridGC.lines:
    line.active = True
options = gce.PowerFlowOptions(gce.SolverType.NR, initialize_with_existing_solution=False,control_q=False, verbose=False)
power_flowPP2GC = gce.PowerFlowDriver(gridGC, options)
power_flowPP2GC.run()
print("   ", power_flowPP2GC.results.get_bus_df().tail(4))
#print("   ", power_flowPP2GC.results.get_branch_df().tail(1))
print("   power losses:", power_flowPP2GC.results.losses.sum())

powerflow of the original pandapower network in pandapower
   bus -5:        vm_pu  va_degree     p_mw   q_mvar
29 0.921950   0.495585 0.200000 0.600000
30 0.917789   0.411178 0.150000 0.070000
31 0.916873   0.388135 0.210000 0.100000
32 0.916590   0.380405 0.060000 0.040000
   power losses: 0.20267711266924443 0.135140961757553
             Vm       Va         P         Q
30 0.921950 0.495586 -0.200000 -0.600000
31 0.917789 0.411178 -0.150000 -0.070000
32 0.916873 0.388135 -0.210000 -0.100000
33 0.916590 0.380405 -0.060000 -0.040000
   power losses: (0.20267711696929647+0.13514096425770317j)


In [197]:
gridPP=nw.case118()

pp.runpp(gridPP)
print("powerflow of the original pandapower network in pandapower")
print("   bus -5: ", gridPP.res_bus.tail(4))
#print("   line -5: ", gridPP.res_line.tail(1))
print("   power losses:", gridPP.res_line.pl_mw.sum(), gridPP.res_line.ql_mvar.sum())

gridGC = GC_PandaPowerImporter.PP2GC(gridPP)

for line in gridGC.lines:
    line.active = True
options = gce.PowerFlowOptions(gce.SolverType.NR, initialize_with_existing_solution=False,control_q=False, verbose=False)
power_flowPP2GC = gce.PowerFlowDriver(gridGC, options)
power_flowPP2GC.run()
print("   ", power_flowPP2GC.results.get_bus_df().tail(4))
#print("   ", power_flowPP2GC.results.get_branch_df().tail(1))
print("   power losses:", power_flowPP2GC.results.losses.sum())

powerflow of the original pandapower network in pandapower
   bus -5:         vm_pu  va_degree       p_mw      q_mvar
114 0.960023  14.690971  22.000000    7.000000
115 1.005000  27.111814 184.000000 -213.043755
116 0.973824  10.916680  20.000000    8.000000
117 0.949438  21.949671  33.000000   15.000000
   power losses: 132.6426306204937 -502.63003062804245
              Vm         Va           P          Q
115 0.960023 -15.308634  -22.000000  -7.000000
116 1.005000  -2.887633 -184.000000 213.043154
117 0.973824 -19.082886  -20.000000  -8.000000
118 0.949438  -8.050119  -33.000000 -15.000000
   power losses: (133.12582813777644-228.0350668368595j)


In [198]:
gridPP=nw.case2848rte()

pp.runpp(gridPP)
print("powerflow of the original pandapower network in pandapower")
print("   bus -5: ", gridPP.res_bus.tail(4))
#print("   line -5: ", gridPP.res_line.tail(1))
print("   power losses:", gridPP.res_line.pl_mw.sum(), gridPP.res_line.ql_mvar.sum())

gridGC = GC_PandaPowerImporter.PP2GC(gridPP)

for line in gridGC.lines:
    line.active = True
options = gce.PowerFlowOptions(gce.SolverType.NR, initialize_with_existing_solution=False,control_q=False, verbose=False)
power_flowPP2GC = gce.PowerFlowDriver(gridGC, options)
power_flowPP2GC.run()
print("   ", power_flowPP2GC.results.get_bus_df().tail(4))
#print("   ", power_flowPP2GC.results.get_branch_df().tail(1))
print("   power losses:", power_flowPP2GC.results.losses.sum())

powerflow of the original pandapower network in pandapower
   bus -5:          vm_pu  va_degree       p_mw     q_mvar
2844 1.060227  -6.411585 107.500000 -10.600000
2845 1.060229  -6.411657 107.400000 -10.600000
2846 1.067666  -5.572330 270.100000 -82.500000
2847 1.040056 -16.991896 615.700000 -55.800000
   power losses: 594.7104303057665 -14783.368533582066
               Vm         Va           P         Q
2845 1.059764  -2.758733 -107.500000 10.600000
2846 1.059766  -2.758806 -107.400000 10.600000
2847 1.067012  -1.918926 -270.100000 82.500000
2848 1.031003 -13.511934 -615.700000 55.800000
   power losses: (710.2528583411859-7717.858334649208j)


In [199]:
sb_code1 = "1-HVMV-urban-2.203-0-no_sw"
gridPP = sb.get_simbench_net(sb_code1)
gridPP.switch.drop([232,234,236,238,240, 242,244,246], inplace=True)
gridPP.ext_grid.at[0,'name']="grid_ext"
gridPP.line['in_service'] = True

#print(gridPP.ext_grid)
#print(gridPP.bus.loc[gridPP.ext_grid.bus])

#pp.topology.create_nxgraph(gridPP)
#print(pp.topology.connected_components(gridPP))

#print(gridPP.switch.head())
#print(len(gridPP.switch))

#pp.diagnostic(net=gridPP)

pp.runpp(gridPP)
print("powerflow of the original pandapower network in pandapower")
print("   bus -5: ", gridPP.res_bus.tail(4))
print("   line -5: ", gridPP.res_line.tail(1))
print("   power losses:", gridPP.res_line.pl_mw.sum(), gridPP.res_line.ql_mvar.sum())

gridGC = GC_PandaPowerImporter.PP2GC(gridPP)

for line in gridGC.lines:
    line.active = True
options = gce.PowerFlowOptions(gce.SolverType.NR, initialize_with_existing_solution=False,control_q=False, verbose=False)
power_flowPP2GC = gce.PowerFlowDriver(gridGC, options)
power_flowPP2GC.run()
print("   ", power_flowPP2GC.results.get_bus_df().tail(4))
print("   ", power_flowPP2GC.results.get_branch_df().tail(1))
print("   power losses:", power_flowPP2GC.results.losses.sum())

LoadflowNotConverged: Power Flow nr did not converge after 10 iterations!

In [57]:
import pandapower as pp
import networkx as nx

# 1)确保已经加载好网： gridPP
G = pp.topology.create_nxgraph(gridPP)
cc = list(nx.connected_components(G))
print("Number of connected components:", len(cc))
for i, comp in enumerate(cc):
    print(f"component {i}: total buses = {len(comp)}; example buses = {sorted(list(comp))[:20]}")
# ext_grid 所在分量
ext_bus = int(gridPP.ext_grid.at[0,'bus'])
for i, comp in enumerate(cc):
    if ext_bus in comp:
        print("ext_grid (bus {}) is in component {}".format(ext_bus, i))
        break
else:
    print("ext_grid's bus not found in any component!")

# 2)检查 r/x/length 是否为 0 或非常小
lines = gridPP.line.copy()
rcol = 'r_ohm_per_km' if 'r_ohm_per_km' in lines.columns else 'r_ohm_per_km'
xcol = 'x_ohm_per_km' if 'x_ohm_per_km' in lines.columns else 'x_ohm_per_km'
lencol = 'length_km' if 'length_km' in lines.columns else 'length_km'

zero_imp = lines[(lines[rcol].fillna(0).abs() < 1e-12) & (lines[xcol].fillna(0).abs() < 1e-12)]
zero_len = lines[lines[lencol].fillna(1e9) == 0]
very_small = lines[(lines[rcol].fillna(0).abs() < 1e-6) & (lines[xcol].fillna(0).abs() < 1e-6)]

print("Lines with r==0 & x==0:", zero_imp.index.tolist())
print("Lines with length == 0:", zero_len.index.tolist())
print("Lines with very small r/x:", very_small.index.tolist()[:80])

# 3)检查变压器参数
print(gridPP.trafo[['vn_hv_kv','vn_lv_kv','vk_percent','vkr_percent']])
print("Transformers with vk_percent > 15:", gridPP.trafo[gridPP.trafo['vk_percent']>15].index.tolist())

# 4)检查是否存在孤立母线
connected_buses = set()
if 'from_bus' in gridPP.line.columns and 'to_bus' in gridPP.line.columns:
    connected_buses |= set(gridPP.line['from_bus'].unique())
    connected_buses |= set(gridPP.line['to_bus'].unique())
if 'hv_bus' in gridPP.trafo.columns:
    connected_buses |= set(gridPP.trafo['hv_bus'].unique())
if 'lv_bus' in gridPP.trafo.columns:
    connected_buses |= set(gridPP.trafo['lv_bus'].unique())

isolated = sorted(list(set(gridPP.bus.index) - connected_buses))
print("Number of isolated buses (no line/trafo connection):", len(isolated))
print("Some isolated buses:", isolated[:80])

# 5)列出 in_service=False 的线路/变压器
print("Lines out-of-service count:", int((gridPP.line['in_service']==False).sum()))
print("Example lines out-of-service:", gridPP.line[gridPP.line['in_service']==False].index.tolist()[:40])
print("Trafos out-of-service count:", int((gridPP.trafo['in_service']==False).sum()))
print("Example trafos out-of-service:", gridPP.trafo[gridPP.trafo['in_service']==False].index.tolist()[:40])



Number of connected components: 1
component 0: total buses = 196; example buses = [np.int64(0), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(12), np.int64(13), np.int64(14), np.int64(15), np.int64(16), np.int64(17), np.int64(18), np.int64(19), np.int64(20)]
ext_grid (bus 361) is in component 0
Lines with r==0 & x==0: []
Lines with length == 0: []
Lines with very small r/x: []
    vn_hv_kv   vn_lv_kv  vk_percent  vkr_percent
0 110.000000  20.000000   16.200000     0.340000
1 110.000000  20.000000   16.200000     0.340000
2 220.000000 110.000000   12.000000     0.128333
3 220.000000 110.000000   12.000000     0.128333
4 220.000000 110.000000   12.000000     0.128333
Transformers with vk_percent > 15: [0, 1]
Number of isolated buses (no line/trafo connection): 0
Some isolated buses: []
Lines out-of-service count: 0
Example lines out-of-service: []
Trafos out-of-service count: 0
Example trafos o

In [59]:
# 1) 先检查注入
print("loads: count", len(gridPP.load), " total P MW:", gridPP.load['p_mw'].sum(), " min/max P_mw:", gridPP.load['p_mw'].min(), gridPP.load['p_mw'].max())
print(gridPP.load[['bus','p_mw','q_mvar']].sort_values('p_mw', ascending=False).head(10))

print("sgen: count", len(gridPP.sgen), " total P MW:", gridPP.sgen['p_mw'].sum(), " min/max P_mw:", gridPP.sgen['p_mw'].min(), gridPP.sgen['p_mw'].max())
print(gridPP.sgen[['bus','p_mw','q_mvar']].sort_values('p_mw', ascending=False).head(10))

print("gen: count", len(gridPP.gen), " total P MW:", gridPP.gen['p_mw'].sum() if len(gridPP.gen)>0 else 0)
print("ext_grid entries:", gridPP.ext_grid)

loads: count 194  total P MW: 521.7199999999999  min/max P_mw: 0.08 49.71
     bus      p_mw    q_mvar
193  560 49.710000 19.647000
192  448 49.710000 19.647000
181  526 34.480000 13.627000
182  528 34.480000 13.627000
183  643 34.480000 13.627000
189  451 31.640000 12.505000
190  675 31.640000 12.505000
191  655 31.640000 12.505000
186  569 17.260000  6.822000
184  424 17.260000  6.822000
sgen: count 219  total P MW: 609.9989999999999  min/max P_mw: 0.0 46.22
     bus      p_mw   q_mvar
215  451 46.220000 0.000000
208  414 33.480000 0.000000
217  619 33.480000 0.000000
190  424 25.570000 0.000000
192  569 25.570000 0.000000
193  669 25.570000 0.000000
191  461 25.570000 0.000000
194  712 25.570000 0.000000
212  586 25.070000 0.000000
196  675 23.800000 0.000000
gen: count 0  total P MW: 0
ext_grid entries:        name  bus    vm_pu  va_degree  slack_weight  in_service controllable p_disp_mw type max_q_mvar min_q_mvar min_p_mw max_p_mw       subnet profile voltLvl    phys_type sn_mva
0

In [61]:
# 2)立即尝试两种不同的潮流设置
import pandapower as pp

# 尝试 A: NR + DC init + 更大迭代次数
try:
    pp.runpp(gridPP, algorithm='nr', init='dc', max_iteration=200, tolerance_mva=1e-8)
    print("nr+dc converged")
except Exception as e:
    print("nr+dc failed:", repr(e))

# 尝试 B: BFSW（配电网常用）
try:
    pp.runpp(gridPP, algorithm='bfsw')
    print("bfsw converged")
except Exception as e:
    print("bfsw failed:", repr(e))

nr+dc failed: LoadflowNotConverged('Power Flow nr did not converge after 200 iterations!')
bfsw failed: ValueError('negative axis 1 index: -1')


In [62]:
# 3) 临时修改 trafo 的 vk_percent
# 备份当前值
orig_vk = gridPP.trafo['vk_percent'].copy()
# 将所有 vk_percent > 15 的 trafo 设置为 15（临时）
gridPP.trafo.loc[gridPP.trafo['vk_percent']>15, 'vk_percent'] = 15.0
print("changed vk_percent for trafos:", gridPP.trafo.loc[gridPP.trafo['vk_percent']==15].index.tolist())

# 再跑一次 nr+dc 尝试
import pandapower as pp
try:
    pp.runpp(gridPP, algorithm='nr', init='dc', max_iteration=200, tolerance_mva=1e-8)
    print("nr+dc converged after lowering vk_percent")
except Exception as e:
    print("nr+dc still failed after lowering vk_percent:", repr(e))

# 恢复原始值（无论成功与否）
gridPP.trafo['vk_percent'] = orig_vk

changed vk_percent for trafos: [0, 1]
nr+dc still failed after lowering vk_percent: LoadflowNotConverged('Power Flow nr did not converge after 200 iterations!')


In [63]:
# 0 injection
# 备份
loads_p = gridPP.load['p_mw'].copy()
loads_q = gridPP.load['q_mvar'].copy()
sgen_p  = gridPP.sgen['p_mw'].copy()
sgen_q  = gridPP.sgen['q_mvar'].copy()

# 将所有负荷与分布式发电置 0（临时）
gridPP.load.loc[:, ['p_mw','q_mvar']] = 0.0
gridPP.sgen.loc[:, ['p_mw','q_mvar']] = 0.0

# 运行潮流（NR + dc init, 更大迭代）
import pandapower as pp
try:
    pp.runpp(gridPP, algorithm='nr', init='dc', max_iteration=200, tolerance_mva=1e-8)
    print("ZERO-INJECTION runpp: CONVERGED")
except Exception as e:
    print("ZERO-INJECTION runpp: FAILED ->", repr(e))

# 恢复
gridPP.load['p_mw'] = loads_p
gridPP.load['q_mvar'] = loads_q
gridPP.sgen['p_mw'] = sgen_p
gridPP.sgen['q_mvar'] = sgen_q


ZERO-INJECTION runpp: FAILED -> LoadflowNotConverged('Power Flow nr did not converge after 200 iterations!')


In [64]:
# Failed
print("lines r_ohm_per_km min/max:", gridPP.line['r_ohm_per_km'].min(), gridPP.line['r_ohm_per_km'].max())
print("lines x_ohm_per_km min/max:", gridPP.line['x_ohm_per_km'].min(), gridPP.line['x_ohm_per_km'].max())
print("lines length_km min/max:", gridPP.line['length_km'].min(), gridPP.line['length_km'].max())
# show extreme
print(gridPP.line[['from_bus','to_bus','r_ohm_per_km','x_ohm_per_km','length_km']].sort_values('r_ohm_per_km').head(10))
print(gridPP.line[['from_bus','to_bus','r_ohm_per_km','x_ohm_per_km','length_km']].sort_values('x_ohm_per_km').head(10))
print(gridPP.trafo[['hv_bus','lv_bus','vn_hv_kv','vn_lv_kv','vk_percent','vkr_percent','parallel']])
pairs = gridPP.line.groupby(['from_bus','to_bus']).size()
dup_pairs = pairs[pairs>1]
print("Duplicate (from,to) line pairs:", dup_pairs.head(20))


lines r_ohm_per_km min/max: 0.1095 0.5939
lines x_ohm_per_km min/max: 0.119 0.372
lines length_km min/max: 0.15 51.636
     from_bus  to_bus  r_ohm_per_km  x_ohm_per_km  length_km
121       388     537      0.109500      0.296000   4.682650
122       473     704      0.109500      0.296000   5.839610
123       473     704      0.109500      0.296000   5.839610
127       424     430      0.109500      0.296000   3.192820
220       398     586      0.109500      0.296000   7.133880
219       443     619      0.109500      0.296000   7.332100
217       560     455      0.109500      0.296000   7.122820
222       408     473      0.109500      0.296000   8.854110
216       661     678      0.109500      0.296000  11.166600
211       398     458      0.109500      0.296000  30.555000
     from_bus  to_bus  r_ohm_per_km  x_ohm_per_km  length_km
23         24      25      0.253000      0.119000   0.500000
24         25      26      0.253000      0.119000   0.350000
22          0      24      

In [65]:
# 方案 A 一次性临时禁用“重复对”中的额外线路与多余并联变压器（快速验证）
import pandapower as pp
from collections import defaultdict

# 备份
line_in_service_backup = gridPP.line['in_service'].copy()
trafo_in_service_backup = gridPP.trafo['in_service'].copy()

# 找到重复 (from_bus, to_bus) 对（只考虑有重复的）
pairs = gridPP.line.groupby(['from_bus','to_bus']).apply(lambda df: list(df.index)).to_dict()
dup_pairs = {k:v for k,v in pairs.items() if len(v)>1}

# 将每个重复对中，保留第一个，其他设为 in_service=False
disabled_lines = []
for k, idx_list in dup_pairs.items():
    for idx in idx_list[1:]:
        gridPP.line.at[idx, 'in_service'] = False
        disabled_lines.append(idx)

# 对 trafo：找 hv_bus/lv_bus 重复的（并联 trafo）
trafo_pairs = gridPP.trafo.groupby(['hv_bus','lv_bus']).apply(lambda df: list(df.index)).to_dict()
dup_trafos = {k:v for k,v in trafo_pairs.items() if len(v)>1}

disabled_trafos = []
for k, idx_list in dup_trafos.items():
    for idx in idx_list[1:]:
        gridPP.trafo.at[idx, 'in_service'] = False
        disabled_trafos.append(idx)

print("Temporarily disabled line indices (duplicates):", disabled_lines[:80])
print("Temporarily disabled trafo indices (duplicates):", disabled_trafos)

# 运行潮流
try:
    pp.runpp(gridPP, algorithm='nr', init='dc', max_iteration=200, tolerance_mva=1e-8)
    print("runpp succeeded AFTER disabling duplicate extra branches.")
    conv_success = True
except Exception as e:
    print("runpp STILL failed after disabling duplicates:", repr(e))
    conv_success = False

# 恢复原始 in_service 状态（无论成功与否）
gridPP.line['in_service'] = line_in_service_backup
gridPP.trafo['in_service'] = trafo_in_service_backup

print("Restored original in_service flags.")


Temporarily disabled line indices (duplicates): [233, 208, 227, 232, 140, 225, 231, 151, 186, 173, 123, 226, 187, 178, 170, 139, 166, 161, 228, 188, 229, 180, 223, 230]
Temporarily disabled trafo indices (duplicates): [3, 4, 1]
runpp STILL failed after disabling duplicates: LoadflowNotConverged('Power Flow nr did not converge after 200 iterations!')
Restored original in_service flags.


In [66]:
# 方案 B 若一次性禁用所有重复额外分支仍不收敛，逐对测试哪些具体重复对触发了问题。下面脚本会对每一对重复（线或 trafo）单独禁用它的额外条目并测试，输出每对测试结果，从而定位出“有问题”的具体对。
import pandapower as pp

# 备份
line_backup = gridPP.line['in_service'].copy()
trafo_backup = gridPP.trafo['in_service'].copy()

problematic_line_pairs = []
# 对每个重复的 line pair/group 单独测试
pairs = gridPP.line.groupby(['from_bus','to_bus']).apply(lambda df: list(df.index)).to_dict()
dup_pairs = {k:v for k,v in pairs.items() if len(v)>1}

for k, idx_list in dup_pairs.items():
    # disable all extras for this pair only, leave others as original
    gridPP.line['in_service'] = line_backup.copy()
    for idx in idx_list[1:]:
        gridPP.line.at[idx, 'in_service'] = False
    try:
        pp.runpp(gridPP, algorithm='nr', init='dc', max_iteration=200, tolerance_mva=1e-8)
        ok = True
    except Exception as e:
        ok = False
    print(f"Testing pair {k}, indices {idx_list} -> converged? {ok}")
    if not ok:
        problematic_line_pairs.append((k, idx_list))

# restore
gridPP.line['in_service'] = line_backup

# 然后对 trafo 的并联组做同样测试
problematic_trafo_pairs = []
trafo_pairs = gridPP.trafo.groupby(['hv_bus','lv_bus']).apply(lambda df: list(df.index)).to_dict()
dup_trafos = {k:v for k,v in trafo_pairs.items() if len(v)>1}

for k, idx_list in dup_trafos.items():
    gridPP.trafo['in_service'] = trafo_backup.copy()
    for idx in idx_list[1:]:
        gridPP.trafo.at[idx, 'in_service'] = False
    try:
        pp.runpp(gridPP, algorithm='nr', init='dc', max_iteration=200, tolerance_mva=1e-8)
        ok = True
    except Exception as e:
        ok = False
    print(f"Testing trafo group {k}, indices {idx_list} -> converged? {ok}")
    if not ok:
        problematic_trafo_pairs.append((k, idx_list))

# restore
gridPP.trafo['in_service'] = trafo_backup

print("Problematic line pairs (that STILL fail even when extras disabled):", problematic_line_pairs)
print("Problematic trafo groups:", problematic_trafo_pairs)


Testing pair (372, 418), indices [133, 233] -> converged? False
Testing pair (372, 655), indices [124, 208] -> converged? False
Testing pair (372, 704), indices [143, 227, 232] -> converged? False
Testing pair (392, 571), indices [136, 140] -> converged? False
Testing pair (392, 594), indices [134, 225] -> converged? False
Testing pair (418, 424), indices [181, 231] -> converged? False
Testing pair (424, 430), indices [127, 151] -> converged? False
Testing pair (430, 520), indices [125, 186] -> converged? False
Testing pair (473, 601), indices [129, 173] -> converged? False
Testing pair (473, 704), indices [122, 123, 226] -> converged? False
Testing pair (506, 526), indices [132, 187] -> converged? False
Testing pair (506, 645), indices [177, 178] -> converged? False
Testing pair (520, 526), indices [128, 170] -> converged? False
Testing pair (540, 594), indices [131, 139] -> converged? False
Testing pair (540, 601), indices [130, 166] -> converged? False
Testing pair (552, 630), indic

# Using Simbench time series

In [71]:
def save_dict_progressively(data, filename):
    with open(filename, 'w') as f:
        json.dump(data, f)

def load_dict(filename):
    with open(filename, 'r') as f:
        return json.load(f)

# define a function to apply absolute values
def apply_absolute_values(gridPP, absolute_values_dict, case_or_time_step):
    for elm_param in absolute_values_dict.keys():
        if absolute_values_dict[elm_param].shape[1]:
            elm = elm_param[0]
            param = elm_param[1]
            gridPP[elm].loc[:, param] = absolute_values_dict[elm_param].loc[case_or_time_step]


In [72]:
caseName = "1-HVMV-urban-2.203-0-no_sw"
sb_code1 = "1-HVMV-urban-2.203-0-no_sw"
import simbench as sb
print(f"loading the example network: {sb_code1}")
gridPP = sb.get_simbench_net(sb_code1)
#delete duplicate lines and trafos, and switches, to avoid issues with GC conversion
gridPP.switch.drop([232,234,236,238,240, 242,244,246], inplace=True)
gridPP.trafo.drop([1,3,4], inplace=True)
gridPP.line.drop(set([123,226,139,140,151,161,166,170,173,178,180,186,187,188,208,223,225,123,226,227,232,228,229,230,231,227,232,233]), inplace=True)
gridPP.ext_grid.at[0,'name']="grid_ext"
gridPP.line['in_service'] = True
#run pandapower powerflow
pp.runpp(gridPP)
#import the network to gridcal
gridGC = GC_PandaPowerImporter.PP2GC(gridPP)


loading the example network: 1-HVMV-urban-2.203-0-no_sw


LoadflowNotConverged: Power Flow nr did not converge after 10 iterations!

In [73]:

profiles = sb.get_absolute_values(gridPP, profiles_instead_of_study_cases=True)
# check that all needed profiles existent
assert not sb.profiles_are_missing(gridPP)


In [74]:
os.getcwd()

'E:\\202510\\VeraGrid'

In [75]:
# Define the start date (January 1st of the year)
start_date = '2024-01-01 00:00:00'
# Create a date range with 15-minute intervals for a year
timestamps = pd.date_range(start=start_date, periods=96 * 366, freq='15T')
total_n_steps = len(profiles[('load', 'q_mvar')])
hours = total_n_steps / 4
days = hours / 24
samples_per_day = 4*24
print(f"A total of {total_n_steps} samples which means {hours} hours and {days} days, from {timestamps[0]} to {timestamps[-1]}")

first_day = 90  #10,90,180,270,350
days = 2
sample_coef = 4 #uses one of every sample_coef samples (i.e if sample_coef=4 uses one of each 4 samples = 1 sample/hour)
TimeRange = int(days * samples_per_day/sample_coef)
initial_sample = samples_per_day*first_day
print(f"the selected time span of {days}, taking 1 of each {sample_coef} samples, with a total of {TimeRange} samples to analyze, with initial sample = {initial_sample}")

# time range calculation
time_steps = range(initial_sample,initial_sample+TimeRange+1)  #96*10)
print(time_steps,time_steps[0], time_steps[-1], time_steps[-1] - time_steps[0])


# run the time series and show the power losses at each step
results = {}
for time_step in time_steps:
    #charges the new profile and converts into GC grid
    apply_absolute_values(gridPP, profiles, time_step)
    gridGC = GC_PandaPowerImporter.PP2GC(gridPP)
    loss_init = sum(runpp(gridGC).losses).real
    print(f"    timestep: {time_step}/{time_steps[-1]} --  powerflow: {loss_init}   ")    

A total of 35136 samples which means 8784.0 hours and 366.0 days, from 2024-01-01 00:00:00 to 2024-12-31 23:45:00
the selected time span of 2, taking 1 of each 4 samples, with a total of 48 samples to analyze, with initial sample = 8640
range(8640, 8689) 8640 8688 48
    timestep: 8640/8688 --  powerflow: 0.27679861643945247   
    timestep: 8641/8688 --  powerflow: 0.25127985432622335   
    timestep: 8642/8688 --  powerflow: 0.2703561541678122   
    timestep: 8643/8688 --  powerflow: 0.26796399286307093   
    timestep: 8644/8688 --  powerflow: 0.2546255576051121   
    timestep: 8645/8688 --  powerflow: 0.2694510369256786   
    timestep: 8646/8688 --  powerflow: 0.2525072381675547   
    timestep: 8647/8688 --  powerflow: 0.24870177151830186   
    timestep: 8648/8688 --  powerflow: 0.2474380887719925   
    timestep: 8649/8688 --  powerflow: 0.2671254094673369   
    timestep: 8650/8688 --  powerflow: 0.2705757716267814   
    timestep: 8651/8688 --  powerflow: 0.2875721060734193

# Optimal Power Flow

In [76]:
#load the network
fname = '.\\networks\\case30.m'
#fname = 'networks//case89pegase.m'
main_circuit = gce.FileOpen(fname).open()

In [77]:
#print the generators information
print("Total generators", sum([generator.P for generator in main_circuit.generators]))

for idx, gen in enumerate(main_circuit.generators):
    print(gen.bus, gen.Cost0, gen.Cost, gen.Cost2, gen.Pmax, gen.Pmin, gen.Qmax, gen.Qmin, gen.P)

Total generators 189.20999999999998
1 0.0 2.0 0.02 80.0 0.0 150.0 -20.0 23.54
2 0.0 1.75 0.0175 80.0 0.0 60.0 -20.0 60.97
22 0.0 1.0 0.0625 50.0 0.0 62.5 -15.0 21.59
27 0.0 3.25 0.00834 55.0 0.0 48.7 -15.0 26.91
23 0.0 3.0 0.025 30.0 0.0 40.0 -10.0 19.2
13 0.0 3.0 0.025 40.0 0.0 44.7 -15.0 37.0


In [78]:
# execute the power flow from the initial setup
pf_options = gce.PowerFlowOptions(solver_type=gce.SolverType.NR)
pf_driver = gce.PowerFlowDriver(grid=main_circuit, options=pf_options)
pf_driver.run()
# Print results
print("convergence:", pf_driver.results.converged)
print(f"Total P,Q load : {sum([load.P for load in main_circuit.loads]), sum([load.Q for load in main_circuit.loads])}")
print(f"Total P generators : {sum([generator.P for generator in main_circuit.generators]):.3f}")
print(f"losses : {pf_driver.results.get_branch_df().Ploss.sum()}")

convergence: True
Total P,Q load : (189.19999999999996, 107.20000000000002)
Total P generators : 189.210
losses : 2.443803129741428


In [79]:
#Let's calculate the OPF for a time snapshot with the linear solver
LIN_opf_driver = gce.OptimalPowerFlowDriver(grid=main_circuit)
LIN_opf_driver.run()
# create the power flow driver, with the OPF results
LIN_pf_options = gce.PowerFlowOptions(solver_type=gce.SolverType.NR)
LIN_pf_driver = gce.PowerFlowDriver(grid=main_circuit,
                                options=LIN_pf_options,
                                opf_results=LIN_opf_driver.results)
LIN_pf_driver.run()

# Print results
print("convergence:", pf_driver.results.converged)
print(f"Total P,Q load : {sum([load.P for load in main_circuit.loads]), sum([load.Q for load in main_circuit.loads])}")
print(f"Total P generators : {LIN_opf_driver.results.generator_power.sum():.3f}")
print(f"losses : {LIN_pf_driver.results.get_branch_df().Ploss.sum():.3f}")

convergence: True
Total P,Q load : (189.19999999999996, 107.20000000000002)
Total P generators : 189.200
losses : 4.592


In [80]:
LIN_pf_driver.results.get_branch_df()

,Pf,Qf,Pt,Qt,loading,Ploss,Qloss
1_2_1,30.218948,-11.236985,-30.017349,8.841781,23.245344,0.201599,-2.395204
1_3_1,31.875904,4.205785,-31.354317,-4.176124,24.519926,0.521587,0.029661
2_4_1,31.484231,5.240841,-30.866108,-5.433841,48.437278,0.618123,-0.193000
3_4_1,28.954316,2.976124,-28.865358,-2.620292,22.272551,0.088958,0.355833
2_5_1,20.856670,4.644745,-20.623238,-5.669566,16.043592,0.233432,-1.024821
2_6_1,35.976448,7.319434,-35.158338,-6.796889,55.348382,0.818111,0.522545
4_6_1,27.371705,9.045660,-27.283704,-8.693656,30.413006,0.088001,0.352003
5_7_1,20.623237,5.851690,-20.380475,-6.210110,29.461768,0.242762,-0.358420
6_7_1,2.427233,3.782777,-2.419525,-4.689890,1.867103,0.007709,-0.907113
6_8_1,28.985290,24.256742,-28.831979,-23.643496,90.579032,0.153312,0.613246


In [81]:
#Let's calculate the OPF for a time snapshot with the non linear solver
solver = gce.SolverType.NONLINEAR_OPF
mip_solver = gce.MIPSolvers.HIGHS
grouping = gce.TimeGrouping.Daily
nl_opf_options = gce.PowerFlowOptions()

options3 = gce.OptimalPowerFlowOptions(solver=solver,
                                  time_grouping=grouping,
                                  mip_solver=mip_solver,
                                  power_flow_options=nl_opf_options)

# create the OPF time instance
NL_opf = gce.OptimalPowerFlowDriver(grid=main_circuit, options=options3)
NL_opf.run()

# create the power flow driver, with the OPF results
NL_pf_options = gce.PowerFlowOptions(solver_type=gce.SolverType.NR)
NL_pf_driver = gce.PowerFlowDriver( grid=main_circuit,
                                    options=NL_pf_options,
                                    opf_results=NL_opf.results)
NL_pf_driver.run()
print("convergence:", pf_driver.results.converged)
print("Total load", sum([load.P for load in main_circuit.loads]), sum([load.Q for load in main_circuit.loads]))
print("total power=", NL_opf.results.generator_power.sum())
print("losses",NL_pf_driver.results.get_branch_df().Ploss.sum())


convergence: True
Total load 189.19999999999996 107.20000000000002
total power= 192.05698251972532
losses 2.9118189060429085


In [82]:
NL_pf_driver.results.get_branch_df()[['Pf','Qf']]

,Pf,Qf
1_2_1,21.586198,-8.523634
1_3_1,19.976229,3.895392
2_4_1,18.793400,5.523783
3_4_1,17.364722,3.854718
2_5_1,14.501376,4.647393
2_6_1,21.848657,7.692189
4_6_1,17.975927,10.694477
5_7_1,14.380284,6.310114
6_7_1,8.578314,3.089919
6_8_1,23.378783,24.422405


# State Estimation

In [134]:
#Example based on the network proposed by "Power System State Estimation" by Ali Abur and Antonio Gómez Expósito
#it is also the same that is being used on the examples in PandaPower

In [84]:
#setup, execute and return the results of a power flow
def ExecutePF(grid, show=False):
    options = gce.PowerFlowOptions(gce.SolverType.NR, verbose=False)
    power_flow = gce.PowerFlowDriver(grid, options)
    power_flow.run()


    losses = power_flow.results.losses.sum()
    bus_df = power_flow.results.get_bus_df()
    branch_df = power_flow.results.get_branch_df()
    if show:
        print('Converged:', power_flow.results.converged, 'error:', power_flow.results.error)
        print(bus_df)
        print(branch_df)

    return losses, bus_df, branch_df

In [85]:
#creates the network defined in the network propsoed by "Power System State Estimation" by Ali Abur and Antonio Gómez Expósito
#in order to study the state estimation problem
#parameter load when True, adds the loads, aimed to calculate the powerflow
#when False, it creates the network without loads, aimed to calculate the optimal power flow, as the loads will be "substitute" by the measurements

def CreateGrid(load = True):
    grid = gce.MultiCircuit()

    b1 = gce.Bus(name='B1', is_slack=True)
    b2 = gce.Bus(name='B2')
    b3 = gce.Bus(name='B3')

    br1 = gce.Line(bus_from=b1, bus_to=b2, name='Br1', r=0.01, x=0.03)
    br2 = gce.Line(bus_from=b1, bus_to=b3, name='Br2', r=0.02, x=0.05)
    br3 = gce.Line(bus_from=b2, bus_to=b3, name='Br3', r=0.03, x=0.08)

    grid.add_bus(b1)
    grid.add_bus(b2)
    grid.add_bus(b3)

    grid.add_line(br1)
    grid.add_line(br2)
    grid.add_line(br3)
    
    if load:
        load1 = gce.Load('load 1'	, P=50, Q=30	)
        load2 = gce.Load('load 2'	, P=150, Q=80	)
        grid.add_load(b2 ,load1)
        grid.add_load(b3 ,load2)

    return grid


In [86]:
#Create an execute the powerflow of the network with loads
m_circuit = CreateGrid(load=True)
losses, bus_df, branch_df = ExecutePF(m_circuit, show=True)

Converged: True error: 1.2756352418819006e-08
         Vm        Va           P          Q
B1 1.000000  0.000000  205.353755 124.045551
B2 0.974381 -1.241237  -50.000000 -30.000000
B3 0.944011 -2.706174 -149.999999 -79.999999
            Pf        Qf          Pt         Qt      loading    Ploss    Qloss
Br1  89.168693 56.435942  -88.055086 -53.095121  8916.869315 1.113607 3.340821
Br2 116.185062 67.609609 -112.571057 -58.574595 11618.506202 3.614006 9.035014
Br3  38.055086 23.095121  -37.428942 -21.425404  3805.508578 0.626144 1.669716


In [88]:
#Create the network without loads, instead the exampled adds measurements to the network, based on the book "Power System State Estimation" by Ali Abur and Antonio Gómez Expósito
#it executes the State Estimation algorithm implemented in GridCal, and shows the results of the state estimation
m_circuit2 = CreateGrid(load = False)
Sb = 100.0
# Note: THe book measurements are in p.u. so we need to scale them back to insert them
m_circuit2.add_pf_measurement(gce.PfMeasurement(0.888 * Sb, 0.008 * Sb, m_circuit2.lines[0]))
m_circuit2.add_pf_measurement(gce.PfMeasurement(1.173 * Sb, 0.008 * Sb, m_circuit2.lines[1]))
m_circuit2.add_pi_measurement(gce.PiMeasurement(-0.501 * Sb, 0.01 * Sb, m_circuit2.buses[1]))

m_circuit2.add_qf_measurement(gce.QfMeasurement(0.568 * Sb, 0.008 * Sb, m_circuit2.lines[0]))
m_circuit2.add_qf_measurement(gce.QfMeasurement(0.663 * Sb, 0.008 * Sb, m_circuit2.lines[1]))
m_circuit2.add_qi_measurement(gce.QiMeasurement(-0.286 * Sb, 0.01 * Sb, m_circuit2.buses[1]))

m_circuit2.add_vm_measurement(gce.VmMeasurement(1.006, 0.004, m_circuit2.buses[0]))
m_circuit2.add_vm_measurement(gce.VmMeasurement(0.968, 0.004, m_circuit2.buses[1]))


# Declare the simulation driver and run
se = gce.StateEstimation(circuit=m_circuit2)
se.run()

print('Converged:', se.results.converged, 'error:', se.results.error)


Converged: 1 error: 7.869402907090262e-10


In [160]:
#the state estimation output is giving values divided by 100 (Sbase) in version VeraGridEngine 5.5.3. Waiting for the fix in a new release

In [89]:
print("compare the results")
print("---- True Power Flow results ----")
print(bus_df)
print(branch_df)
print("---- State Estimation results ----")
print(se.results.get_bus_df())
print(se.results.get_branch_df())
print("---- error ----")
print(bus_df - se.results.get_bus_df())
print(branch_df - se.results.get_branch_df())
#Calculate the MSE between both measures Powerflow or State Estimation WLS (gridcal), by using the Mean Squared Error (MSE) function 
sepf = se.results.get_branch_df().Pf.values
pfpf = branch_df.Pf.values
error = mean_squared_error(sepf, pfpf)
print(f"Mean Squared Error for Pf: {error}")

seqf = se.results.get_branch_df().Qf.values
pfqf = branch_df.Qf.values
error = mean_squared_error(seqf, pfqf)
print(f"Mean Squared Error for Pf: {error}")

compare the results
---- True Power Flow results ----
         Vm        Va           P          Q
B1 1.000000  0.000000  205.353755 124.045551
B2 0.974381 -1.241237  -50.000000 -30.000000
B3 0.944011 -2.706174 -149.999999 -79.999999
            Pf        Qf          Pt         Qt      loading    Ploss    Qloss
Br1  89.168693 56.435942  -88.055086 -53.095121  8916.869315 1.113607 3.340821
Br2 116.185062 67.609609 -112.571057 -58.574595 11618.506202 3.614006 9.035014
Br3  38.055086 23.095121  -37.428942 -21.425404  3805.508578 0.626144 1.669716
---- State Estimation results ----
         Vm        Va           P          Q
B1 0.999629  0.000000  206.401645 122.644040
B2 0.974156 -1.247547  -49.597497 -29.774953
B3 0.943890 -2.745717 -151.422098 -78.752893
            Pf        Qf          Pt         Qt      loading    Ploss    Qloss
Br1  89.299199 55.882169  -88.188659 -52.550550  8929.919882 1.110540 3.331619
Br2 117.102446 66.761871 -113.465724 -57.670065 11710.244635 3.636722 9.09180

# Contingency analysis

Contingency = outage of an electrical element in the power grid, mainly generators and lines/transfomers
elements = lines, transformers, generators, loads
Contingency N-1 is the usual


In [91]:
#Loading the 5 buses cases from matpower
main_circuit = gce.open_file('.\\networks\\case14.m')

In [92]:
#create lists of branches and generators, which are the elements which can fail, and which will be considered in the Contingency Analysis
branches = main_circuit.get_branches()
gens = main_circuit.get_generators()

In [94]:
# manually generate the contingencies, by adding each branch (branch=line or transformer) and generator to a contingency group
for i, br in enumerate(branches):
    # add a contingency group
    group = gce.ContingencyGroup(name="contingency branch {}".format(i+1))
    main_circuit.add_contingency_group(group)

    # add the branch contingency to the groups, only groups are failed at once
    con = gce.Contingency(device=br, name=br.name, group=group)
    main_circuit.add_contingency(con)
    print("added contingency for branch:",i, br.idtag)

for i, gen in enumerate(gens):
    # add a contingency group
    group = gce.ContingencyGroup(name="contingency gen {}".format(i+1))
    main_circuit.add_contingency_group(group)

    # add the gen contingency to the groups, only groups are failed at once

    con = gce.Contingency(device=gen, name=gen.name, group=group,
                            value=100,
                            prop=gce.ContingencyOperationTypes.PowerPercentage)

    main_circuit.add_contingency(con)
    print("added contingency for gen:",i, gen.idtag)

added contingency for branch: 0 ede8b0e168aa452f8f3ebe4f23b21596
added contingency for branch: 1 6d10e65dd2d042d0982dbdc8bb8b29a1
added contingency for branch: 2 bcf23550118e4fa4b28b6fac1362ac85
added contingency for branch: 3 b85a9323696a4b969101c529f7b2bcd4
added contingency for branch: 4 a57ba9962bdc4ada80afe170ecff7662
added contingency for branch: 5 708ee58fb7504b48aa247f2c07f155ed
added contingency for branch: 6 b175561b44be491982880c7aa09940e0
added contingency for branch: 7 480f464a64c149e5a60d2af3c9f7b5aa
added contingency for branch: 8 d5d4a90894774451bf98dbef844104b3
added contingency for branch: 9 5c0376f1a1424bd1a3e08f2ec94ed343
added contingency for branch: 10 97734e3643834b71bb799092c0f75909
added contingency for branch: 11 9340ed6c53824036862e4942ddc14c0c
added contingency for branch: 12 60b845783db44f6d9d388ae33988e870
added contingency for branch: 13 93cd8895f1c3492cbd5b0076c59ae68e
added contingency for branch: 14 27fefb88f37c47ceb726254aecb87583
added contingency fo

In [95]:
## add some other contingencies, which are considered as specials, i.e. that two lines are off simultaneously, in this lines 3 and 5
group = gce.ContingencyGroup(name="Special contingency two lines off")
main_circuit.add_contingency_group(group)
main_circuit.add_contingency(gce.Contingency(device=branches[3], name=branches[3].name, group=group))
main_circuit.add_contingency(gce.Contingency(device=branches[5], name=branches[5].name, group=group))
#branches[5].name

In [96]:
#the contingency analysis is failing for this circuit in version VeraGridEngine 5.5.3. Waiting for the fix in a new release

In [97]:
#setup and execute the contingency analysis
pf_options = gce.PowerFlowOptions(solver_type=gce.SolverType.NR)
# declare the contingency options
options_ = gce.ContingencyAnalysisOptions(use_provided_flows=False,
                                      Pf=None,
                                      contingency_method=en.ContingencyMethod.PowerFlow,
                                      pf_options=pf_options)

simulation = gce.ContingencyAnalysisDriver(grid=main_circuit,
                                       options=options_,
                                       linear_multiple_contingencies=None  # it is computed inside
                                       )

simulation.run()



In [98]:
# print results
print("Flows:\n", simulation.results.mdl(gce.ResultTypes.BranchActivePowerFrom).to_df())

Flows:
                                          1_2_1      1_5_1     2_3_1     2_4_1      2_5_1      3_4_1       4_5_1     6_11_1    6_12_1    6_13_1     7_8_1     7_9_1    9_10_1    9_14_1    10_11_1   12_13_1    13_14_1     4_7_1     4_9_1     5_6_1
# contingency branch 1                0.000000 260.972616 48.079876  1.793021 -31.572897 -47.150047 -133.795612  10.916564  8.268684 19.603313  0.000000 24.440487  1.744333  7.149219  -7.260777  2.088525   7.927066 24.440487 13.953065 49.988562
# contingency branch 2              240.000070   0.000000 86.766011 83.396870  77.993621 -10.683366  -24.943871   6.153418  7.664435 17.150633  0.000000 29.304968  6.422292 10.142085  -2.592131  1.493646   4.932785 29.304968 16.759410 42.168486
# contingency branch 3              148.264033  95.474163  0.000000 93.742719  68.988877 -94.199998 -101.516989   9.111399  8.019777 18.660474 -0.000000 26.276744  3.502007  8.305061  -5.506481  1.844091   6.764734 26.276744 15.030325 46.991649
# contingenc

In [99]:
print("Report:\n", simulation.results.mdl(gce.ResultTypes.ContingencyAnalysisReport).to_df())

Report:
 Empty DataFrame
Columns: [Time idx, Time, Probability cluster, Area 1, Area 2, Monitored, Contingency, Base rating (MW), Contingency rating (MW), SRAP rating (MW), Base flow (MW), Post-Contingency flow (MW), Post-SRAP flow (MW), Base loading (pu), Post-Contingency loading (pu), Post-SRAP loading (pu), Overload, SRAP availability, SRAP Power (MW), Solved with SRAP]
Index: []


In [100]:
def ExecutePF(grid, show=False):
    options = gce.PowerFlowOptions(gce.SolverType.NR, verbose=False)
    power_flow = gce.PowerFlowDriver(grid, options)
    power_flow.run()


    losses = power_flow.results.losses.sum()
    bus_df = power_flow.results.get_bus_df()
    branch_df = power_flow.results.get_branch_df()
    if show:
        print('Converged:', power_flow.results.converged, 'error:', power_flow.results.error)
        print(losses)
        print(bus_df)
        print(branch_df)

    return losses, bus_df, branch_df

In [101]:
#Let's manually check that the results of the contingency analysis are correct, by running a power flow with one of the contingencies as in the contingency analysis
fname = os.path.join('networks//case14.m')
main_circuit = gce.open_file(fname)

main_circuit.lines[0].active=True
main_circuit.lines[5].active=False

ExecutePF(main_circuit, show=True)

Converged: True error: 7.130622470263859e-08
(13.676836245389916+34.13620758203708j)
         Vm         Va          P          Q
1  1.060000   0.000000 232.676831 -18.337422
2  1.045000  -5.168590  18.300000  33.462725
3  1.010000 -15.695055 -94.200000  11.241325
4  1.020329  -9.365492 -47.799998   3.900003
5  1.021597  -8.132403  -7.599998  -1.599996
6  1.070000 -13.461319 -11.199998   4.020456
7  1.062841 -12.458481   0.000002   0.000004
8  1.090000 -12.458481  -0.000000  16.806020
9  1.057382 -14.063315 -29.500007   4.643091
10 1.052214 -14.242523  -9.000000  -5.799999
11 1.057564 -13.983332  -3.499998  -1.799999
12 1.055278 -14.306869  -6.100000  -1.600000
13 1.050621 -14.380464 -13.499999  -5.800000
14 1.036476 -15.199960 -14.900000  -4.999999
                Pf         Qf          Pt         Qt   loading     Ploss     Qloss
1_2_1   162.508960 -21.709341 -157.893280  29.952442  1.625090  4.615680  8.243102
1_5_1    70.167871   3.371918  -67.782213   1.144804  0.701679  2.385658  

(np.complex128(13.676836245389916+34.13620758203708j),
          Vm         Va          P          Q
 1  1.060000   0.000000 232.676831 -18.337422
 2  1.045000  -5.168590  18.300000  33.462725
 3  1.010000 -15.695055 -94.200000  11.241325
 4  1.020329  -9.365492 -47.799998   3.900003
 5  1.021597  -8.132403  -7.599998  -1.599996
 6  1.070000 -13.461319 -11.199998   4.020456
 7  1.062841 -12.458481   0.000002   0.000004
 8  1.090000 -12.458481  -0.000000  16.806020
 9  1.057382 -14.063315 -29.500007   4.643091
 10 1.052214 -14.242523  -9.000000  -5.799999
 11 1.057564 -13.983332  -3.499998  -1.799999
 12 1.055278 -14.306869  -6.100000  -1.600000
 13 1.050621 -14.380464 -13.499999  -5.800000
 14 1.036476 -15.199960 -14.900000  -4.999999,
                 Pf         Qf          Pt         Qt   loading     Ploss     Qloss
 1_2_1   162.508960 -21.709341 -157.893280  29.952442  1.625090  4.615680  8.243102
 1_5_1    70.167871   3.371918  -67.782213   1.144804  0.701679  2.385658  4.516723
 2

In [103]:

branches = main_circuit.get_branches()

# manually generate the contingencies
for i, br in enumerate(branches):
    # add a contingency group
    group = gce.ContingencyGroup(name="contingency {}".format(i+1))
    main_circuit.add_contingency_group(group)

    # add the branch contingency to the groups, only groups are failed at once
    con = gce.Contingency(device=br, name=br.name, group=group)
    main_circuit.add_contingency(con)

# add a special contingency
group = gce.ContingencyGroup(name="Special contingency")
main_circuit.add_contingency_group(group)
main_circuit.add_contingency(gce.Contingency(device=branches[3], name=branches[3].name, group=group))
main_circuit.add_contingency(gce.Contingency(device=branches[5], name=branches[5].name, group=group))

pf_options = gce.PowerFlowOptions(solver_type=gce.SolverType.NR)

# declare the contingency options
options_ = gce.ContingencyAnalysisOptions(use_provided_flows=False,
                                      Pf=None,
                                      contingency_method=en.ContingencyMethod.PowerFlow,
                                      pf_options=pf_options)

simulation = gce.ContingencyAnalysisDriver(grid=main_circuit,
                                       options=options_,
                                       linear_multiple_contingencies=None  # it is computed inside
                                       )

simulation.run()

# print results
print("Flows:\n", simulation.results.mdl(gce.ResultTypes.BranchActivePowerFrom).to_df())
print("Report:\n", simulation.results.mdl(gce.ResultTypes.ContingencyAnalysisReport).to_df())

Flows:
                            1_2_1      1_5_1     2_3_1      2_4_1      2_5_1      3_4_1       4_5_1     6_11_1    6_12_1    6_13_1     7_8_1     7_9_1    9_10_1    9_14_1    10_11_1   12_13_1    13_14_1     4_7_1     4_9_1     5_6_1
# contingency 1         0.000000 264.948978 98.371209 -25.923606 -54.147603   0.000000 -114.401552  10.181101  8.190252 19.234721  0.000000 25.184583  2.465768  7.586354  -6.538945  2.010971   7.486822 25.184583 14.367539 48.806075
# contingency 2       239.812504   0.000000 98.371209  76.940702  72.672962   0.000000  -20.376966   5.966533  7.642350 17.055472  0.000000 29.497640  6.608166 10.258538  -2.406812  1.471843   4.817419 29.497640 16.869064 41.864355
# contingency 3        76.354659  53.999915  0.000000  51.821962  41.825159   0.000000  -42.821293   6.578015  7.684017 17.346242 -0.000000 28.869954  5.992259  9.921940  -3.023022  1.513838   5.150597 28.869954 16.544244 42.808274
# contingency 4       145.678870  88.726223 98.371209   0.000000